# Generating a single element of the covariance matrix
The first step is to compute the integral for a single pair of objects:

\begin{aligned}
    C^{\rm (v)}_{mn} =& 
     \frac{1}{2\pi^2} \frac{D_1(z_{ m})~D_1(z_{ n})}{D_1^2(0)}\biggl[\frac{H(z_{ m})f(z_{ m})}{(1+z_{ m})}\biggr]  \biggl[\frac{H(z_{ n})f(z_{ n})}{(1+z_{ n})}\biggr] 
   \\& \int \mathrm{d}k W_{mn}(k)\mathcal{Z}(k,z_{ m},z_{ n})P_{\delta}(k,0)  D_{\rm u}^2(k\sigma_{\rm u})\,E(\sigma_8) \, ,
\end{aligned}

where the meaning of each term can be found in Sec. 2 of [our paper](https://arxiv.org/pdf/2504.10453). The most complex part, as you can imagine, is the integral of a heavily-oscillating function as the window function $W_{mn}(k)$. For that we use FFTLog, as implemented in [this repository](https://github.com/xfangcosmo/FFTLog-and-beyond). Make sure you install the code in the repository, as well as [CAMB](https://pypi.org/project/camb/), in order to run this notebook.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
import camb
from camb import model, initialpower
from fftlogx import fftlog
from velocemu.utils import amplitude_W, window_function_w, D1z, prefactor_divergence, B_nonlin_prefac, D_nonlin_prefac,luminosity_distance

In [2]:
# set up a few parameters
kmin = 1e-5 # I assume k in h/Mpc
kmax = 1e2 # I assume k in h/Mpc
nu = 1.0 # for fftlog
N_extrap = 0 # for fftlog

In [3]:
def integrate_fftlog(z1, z2, alpha, Omat, H0, lnAs):
    """Compute a single element of the covariance matrix according to the integral above.
    Parameters
    ----------
    z1 : float
        The redshift of the first object.
    z2 : float
        The redshift of the second object.
    alpha : float
        The angle between the two ojects.
    Omat : float
        The matter density parameter at z=0, usually between 0.1 and 0.5.
    H0 : float
        The Hubble parameter at z=0, usually between 50 and 100.
    lnAs : float
        The natural logarithm of 10^10 A_s, where A_s is the amplitude of the primordial power spectrum.
    """
    # first compute some useful quantities
    As = 1e-10*np.exp(lnAs)
    sigma8 = (0.8102 / (1e-10*np.exp(3.047))) * As # for the non linear corrections
    r1 = luminosity_distance(z1, H0, Omat)
    r2 = luminosity_distance(z2, H0, Omat)
    
    # then we compute the linear matter power spectrum
    # so that we only have to call it once later
    pars = camb.CAMBparams()
    # fix ombh2 = 0.02242, based on Planck 2018 results
    ombh2 = 0.02242
    omch2 = Omat*(H0/100)**2 - ombh2
    pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2)
    pars.InitPower.set_params(ns=0.965, As=As) # k_star default
    # only compute at redshift 0
    pars.set_matter_power(redshifts=np.linspace(0., 2.5, 26), kmax=kmax)
    # linear spectra only
    pars.NonLinear = model.NonLinear_none # the nonlinearities come later
    results = camb.get_results(pars)

    PK = results.get_matter_power_interpolator(nonlinear=False, 
                                               hubble_units=True, # Pk in (Mpc / h)**3
                                               k_hunit=True) # k in h/Mpc

    # terms inside the integral, to compute the unequal-time term
    def integrand_func(k):
        return PK.P(0, k)*B_nonlin_prefac(k, sigma8)*D_nonlin_prefac(k)**2
    k_nl, k_nl_error = integrate.quad(integrand_func, kmin, kmax, limit=100000000, epsabs=1.49e-12, epsrel=1.49e-12)
    k_nl /= 12*np.pi*2 # this is actually k_nl**-2

    # compute integral prefactor
    D1 = D1z(z1, Omat)
    D2 = D1z(z2, Omat)
    prefac = prefactor_divergence(z1, z2, H0, Omat)
    
    # then actually start with the computation
    if r1 == r2 and alpha == 0: # easy case
        def integrand_func(k): # k should be in h/Mpc here
            pk = PK.P(0, k)*B_nonlin_prefac(k, sigma8)*D_nonlin_prefac(k)**2 # redshift = 0, wavelength = k
            # correction factor
            corr = np.exp(-k**2 * k_nl * (D1-D2)**2)
            # window function
            W = window_function_w(k, r1, r2, alpha)
            return pk * corr * W
        final_value, _ = integrate.quad(integrand_func, kmin, kmax, limit=100000000, epsabs=1.49e-12, epsrel=1.49e-12)
    else: # more complex case with FFTLog, we split it in 3
        A = amplitude_W(r1, r2, alpha)
        ks = np.logspace(np.log10(kmin), np.log10(kmax), 100000)        
        def integrand_func_1(k): # k should be in h/Mpc here
            # first part
            g12 = np.cos(alpha) / 3
            pk = PK.P(0, k) *B_nonlin_prefac(k, sigma8)*D_nonlin_prefac(k)**2# redshift = 0, wavelength = k
            corr = np.exp(-k**2 * k_nl * (D1-D2)**2)
            return pk * g12 * corr * k # last multiplication because fftlog divides by variable

        integrand = integrand_func_1(ks)    
        myfftlog = fftlog(ks, integrand, nu=nu, N_extrap_low=N_extrap, N_extrap_high=N_extrap, c_window_width=0.25, N_pad=5000)   
        r, Fr = myfftlog.fftlog(0) # ell=0 is because we consider j0
        # need to interpolate to get precise value, which is A (i.e. amplitude W)
        interpolated_value_1 = np.interp([A], r, Fr)

        def integrand_func_2(k): # k should be in h/Mpc here
            # first part
            g12 = -2 * np.cos(alpha) / 3
            pk = PK.P(0, k) *B_nonlin_prefac(k, sigma8)*D_nonlin_prefac(k)**2# redshift = 0, wavelength = k
            corr = np.exp(-k**2 * k_nl * (D1-D2)**2)
            return pk * g12 * corr * k # last multiplication because fftlog divides by variable

        integrand = integrand_func_2(ks)    
        myfftlog = fftlog(ks, integrand, nu=nu, N_extrap_low=N_extrap, N_extrap_high=N_extrap, c_window_width=0.25, N_pad=5000)   
        r, Fr = myfftlog.fftlog(2) # ell=2 is because we consider j2
        # need to interpolate to get precise value, which is A (i.e. amplitude W)
        interpolated_value_2 = np.interp([A], r, Fr)


        def integrand_func_3(k): # k should be in h/Mpc here
            # first part
            g12 = r1 * r2 * (np.sin(alpha))**2 / A**2
            pk = PK.P(0, k) *B_nonlin_prefac(k, sigma8)*D_nonlin_prefac(k)**2 # redshift = 0, wavelength = k
            corr = np.exp(-k**2 * k_nl * (D1-D2)**2)
            return pk * g12 * corr * k # last multiplication because fftlog divides by variable

        integrand = integrand_func_3(ks)    
        myfftlog = fftlog(ks, integrand, nu=nu, N_extrap_low=N_extrap, N_extrap_high=N_extrap, c_window_width=0.25, N_pad=5000)   
        r, Fr = myfftlog.fftlog(2) # ell=2 is because we consider j2
        # need to interpolate to get precise value, which is A (i.e. amplitude W)
        interpolated_value_3 = np.interp([A], r, Fr)
        
        # then we sum up all values
        final_value = (interpolated_value_1+interpolated_value_2+interpolated_value_3)[0]
    
    # and we multiply by all prefactor terms
    final_value *= prefac * D1 * D2/(2 * np.pi**2 * D1z(0, Omat)**2)
    return final_value

Now let's try it for a pair of objects with redshift $z_1$ and $z_2$, angle $\alpha$, and a cosmological model with $\Omega_\textrm{m}=0.35$, $H_0=70$ km/s/Mpc, and $\log{10^{10}A_\textrm{s}=3.1}$

In [4]:
z1, z2, alpha, Omat, H0, lnAs = 0.2, 0.1, 1., 0.35, 70., 3.1

In [5]:
integral_value = integrate_fftlog(z1, z2, alpha, Omat, H0, lnAs)
print(f'The value of the integral is: {integral_value}')

Note: redshifts have been re-sorted (earliest first)


/tmp/ipykernel_934363/1950520976.py:45: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  k_nl, k_nl_error = integrate.quad(integrand_func, kmin, kmax, limit=100000000, epsabs=1.49e-12, epsrel=1.49e-12)


The value of the integral is: 150.5531238524841


150.5531238524841